# Heroes of Pymoli Data Analysis

### Initial imports and csv read in

In [25]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Load csv file from Resources folder
csvfiledata = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame and prints the header for preview of data.
purchasedatadf = pd.read_csv(csvfiledata)
purchasedatadf.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",$3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,$1.56
2,2,Ithergue48,24,Male,92,Final Critic,$4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,$3.27
4,4,Iskosia90,23,Male,131,Fury,$1.44


In [26]:
#Also posts the shape to get an idea of the data set size
purchasedatadf.shape

(780, 7)

### Player Count

In [27]:
#Counts the unique values in column "SN"
tp = purchasedatadf["SN"].nunique()
tpdf = pd.DataFrame([{"Total Players":tp}])
tpdf

,Total Players
0,576


### Purchasing Analysis (Total)

In [28]:
nuniqueitems = purchasedatadf["Item ID"].nunique()
ttlrevenue = purchasedatadf["Price"].sum()
npurchases = purchasedatadf["Purchase ID"].count()
avgprice = ttlrevenue / npurchases

purchasinganalysis = pd.DataFrame({"Number of Unique Items":[nuniqueitems],
                                   "Average Price":[avgprice],
                                   "Number of Purchases":[npurchases],
                                   "Total Revenue":[ttlrevenue]
                                  })
pd.options.display.float_format = '${:,.2f}'.format
purchasinganalysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


### Gender Demographics

In [29]:
#Removes the rows where a SN is duplicated/a user makes an additional purchase
deduplicateddf = purchasedatadf.drop_duplicates(subset=["SN"], keep='first')
deduplicateddf.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",$3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,$1.56
2,2,Ithergue48,24,Male,92,Final Critic,$4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,$3.27
4,4,Iskosia90,23,Male,131,Fury,$1.44


In [35]:
malecount = deduplicateddf["Gender"] =="Male"
malecountsum = malecount.sum()
femalecount = deduplicateddf["Gender"] == "Female"
femalecountsum = femalecount.sum()
othercount = deduplicateddf["Gender"] == "Other / Non-Disclosed"
othercountsum = othercount.sum()
playersum = deduplicateddf["SN"].count()
genderdf = pd.DataFrame({"Gender":["Male","Female","Other/Non-Disclosed"],
                         "Total Count":[malecountsum,femalecountsum,othercountsum], 
                         "Percentage of Players": [malecountsum / playersum*100, 
                                                   femalecountsum / playersum*100,
                                                   othercountsum / playersum*100]
                         
                        })
pd.options.display.float_format = '{:.2f}%'.format
genderdf.set_index("Gender")


,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


### Purchasing Analysis by Gender

In [37]:
ttlpurchaseval = purchasedatadf.groupby(["Gender"]).sum()["Price"]
ttlpurchasecount = purchasedatadf.groupby(["Gender"]).count()["Price"]
ttlpurchaseid = purchasedatadf["SN"].value_counts()
ttlpurchasevalue = purchasedatadf["Price"].sum()

genderttlcounts = deduplicateddf["Gender"].value_counts()

purchasinganalysisgender = pd.DataFrame({"Purchase Count":ttlpurchasecount,
                                         "Average Purchase Price": ttlpurchaseval / ttlpurchasecount,
                                        "Total Purchase Value":ttlpurchaseval,
                                        "Avg Total Purchase per Person": ttlpurchaseval / genderttlcounts
                                        })

pd.options.display.float_format = '${:,.2f}'.format
purchasinganalysisgender

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age Demographics


In [52]:
bins = [0,9,14,19,24,29,34,39,150]

group_names = ["<10", "10-14", "15-19", "20-24", "25-28", "29-34", "35-39", "40+"]

bindf = purchasedatadf.loc[:,["Age"]]
bindf = bindf.drop_duplicates()
bindf["Ages"] = pd.cut(bindf["Age"], bins, labels=group_names)

agesummaryttl = deduplicateddf["Age Summary"].value_counts()
ageperc = agesummaryttl / tp * 100

agedemosum = pd.DataFrame({"Total Count":agesummaryttl,
                           "Percentage of Players":ageperc})
agedemosum.sort_index()

,Total Count,Percentage of Players
<10,17,$2.95
10-14,22,$3.82
15-19,107,$18.58
20-24,258,$44.79
25-28,77,$13.37
29-34,52,$9.03
35-39,31,$5.38
40+,12,$2.08


### Puchasing Analysis


In [58]:
purchasedatadf["Ages"] = pd.cut(purchasedatadf["Age"], bins, labels=group_names)

ttl = purchasedatadf.groupby(["Ages"]).sum()["Price"]
avg = purchasedatadf.groupby(["Ages"]).mean()["Price"]
cts = purchasedatadf.groupby(["Ages"]).count()["Price"]

purchsum = pd.DataFrame({"Purchase Count": cts,
                         "Average Purchase Price": avg,
                         "Total Purchase": ttl,
                         "Avg Total Purchase per Person": ttl / agedemosum["Total Count"]
                        })
purchsum

,Purchase Count,Average Purchase Price,Total Purchase,Avg Total Purchase per Person
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-28,101,$2.90,$293.00,$3.81
29-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19
<10,23,$3.35,$77.13,$4.54


### Top Spenders


In [62]:
ttl2 = purchasedatadf.groupby(["SN"]).sum()["Price"]
avg2 = purchasedatadf.groupby(["SN"]).mean()["Price"]
cts2 = purchasedatadf.groupby(["SN"]).count()["Price"]

topspendersdf = pd.DataFrame({"Total Purchase Value": ttl2,
                              "Average Purchase Price": avg2,
                              "Purchase Count": cts2
                             })
topspendersdf.sort_values("Total Purchase Value",ascending=False).head(10)

,Total Purchase Value,Average Purchase Price,Purchase Count
SN,,,
Lisosia93,$18.96,$3.79,5
Idastidru52,$15.45,$3.86,4
Chamjask73,$13.83,$4.61,3
Iral74,$13.62,$3.40,4
Iskadarya95,$13.10,$4.37,3
Ilarin91,$12.70,$4.23,3
Ialallo29,$11.84,$3.95,3
Tyidaim51,$11.83,$3.94,3
Lassilsala30,$11.51,$3.84,3


### Most Popular Items

In [64]:
popularitem = purchasedatadf.loc[:,["Item ID","Item Name", "Price"]]

ttl3 = purchasedatadf.groupby(["Item ID","Item Name"]).sum()["Price"]
avg3 = purchasedatadf.groupby(["Item ID","Item Name"]).mean()["Price"]
cts3 = purchasedatadf.groupby(["Item ID","Item Name"]).count()["Price"]

popularitemdf = pd.DataFrame({"Total Purchase Value": ttl3,
                              "Average Purchase Price": avg3,
                              "Purchase Count": cts3
                             })
popularitemdf.sort_values("Purchase Count",ascending=False).head(10)

,,Total Purchase Value,Average Purchase Price,Purchase Count
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$50.76,$4.23,12
145,Fiery Glass Crusader,$41.22,$4.58,9
108,"Extraction, Quickblade Of Trembling Hands",$31.77,$3.53,9
82,Nirvana,$44.10,$4.90,9
19,"Pursuit, Cudgel of Necromancy",$8.16,$1.02,8
103,Singed Scalpel,$34.80,$4.35,8
75,Brutality Ivory Warmace,$19.36,$2.42,8
72,Winter's Bite,$30.16,$3.77,8
60,Wolf,$28.32,$3.54,8


In [65]:
popularitemdf.sort_values("Total Purchase Value", ascending=False).head(10)

,,Total Purchase Value,Average Purchase Price,Purchase Count
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$50.76,$4.23,12
82,Nirvana,$44.10,$4.90,9
145,Fiery Glass Crusader,$41.22,$4.58,9
92,Final Critic,$39.04,$4.88,8
103,Singed Scalpel,$34.80,$4.35,8
59,"Lightning, Etcher of the King",$33.84,$4.23,8
108,"Extraction, Quickblade Of Trembling Hands",$31.77,$3.53,9
78,"Glimmer, Ender of the Moon",$30.80,$4.40,7
72,Winter's Bite,$30.16,$3.77,8
